In [50]:
!git clone https://github.com/muxspace/facial_expressions.git

fatal: destination path 'facial_expressions' already exists and is not an empty directory.


In [51]:
import csv
data={}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2].lower()
    if key in data:
      data[key].append(row[1])
    else :
      data[key]=[row[1]]
    

In [52]:

emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [53]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')


FileExistsError: ignored

In [54]:
os.mkdir('master_data/testing')

FileExistsError: ignored

In [ ]:
for emotion in emotion_list:
  os.mkdir(os.path.join('/content/master_data/training/',emotion))
  os.mkdir(os.path.join('/content/master_data/testing/',emotion))

In [55]:
from shutil import copyfile
split_size=0.8
for emotion,images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/training',emotion,image)
    copyfile(source,dest)
  for image in test_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/testing',emotion,image)
    copyfile(source,dest)

In [56]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout


In [57]:

model=tf.keras.models.Sequential([
    Conv2D(16,(3,3),activation="relu",input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024,activation="relu"),
    Dense(8,activation="softmax") 
])
model.compile(optimizer=Adam(lr=0.01),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 18496)           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [58]:
test_dir='/content/master_data/testing'
train_dir='/content/master_data/training'
train_datagen=ImageDataGenerator(rescale=1/255.0)
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    class_mode="categorical",
    batch_size=128
)
test_datagen=ImageDataGenerator(rescale=1/255.0)
test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    class_mode="categorical",
    batch_size=128
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [59]:
es=EarlyStopping(monitor='val_accuracy',patience=2)

In [60]:
model=model.fit_generator(train_generator,epochs=10,verbose=1,validation_data=test_generator,callbacks=[es])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
86/86 [==============================] - 24s 272ms/step - loss: 1.7616 - accuracy: 0.4774 - val_loss: 1.0315 - val_accuracy: 0.5011
Epoch 2/10
86/86 [==============================] - 23s 269ms/step - loss: 1.0198 - accuracy: 0.5094 - val_loss: 1.0228 - val_accuracy: 0.5047
Epoch 3/10
86/86 [==============================] - 23s 268ms/step - loss: 0.9853 - accuracy: 0.5685 - val_loss: 1.0330 - val_accuracy: 0.5492
Epoch 4/10
86/86 [==============================] - 23s 270ms/step - loss: 0.8898 - accuracy: 0.6583 - val_loss: 1.0033 - val_accuracy: 0.6120
Epoch 5/10
86/86 [==============================] - 23s 265ms/step - loss: 0.8067 - accuracy: 0.7104 - val_loss: 1.0560 - val_accuracy: 0.5598
Epoch 6/10
86/86 [==============================] - 23s 269ms/step - loss: 0.7671 - accuracy: 0.7220 - val_loss: 0.9308 - val_accuracy: 0.6583
Epoch 7/10
86/86 [==============================] - 23s 268ms/step - loss: 0.7080 - accuracy: 0.7512 - val_loss: 1.0343 - val_accuracy: 0.6357